### 멀티클래스에 interpret를 추가하고 VQC(output_shape=num_classes) 
1. output_shape = num_classes
2. interpret = index% num_classes
3. feature=4

In [ ]:
import datetime
import time
import qiskit
import tensorflow 
import platform

datetime.datetime.now()

print('qiskit version:', qiskit.__version__)
print('tf     version:', tensorflow.__version__)
print('Python Version:',platform.python_version())
print('OS            :', platform.system())
print('코딩시간       :', time.strftime("%a %b %d %H:%M:%S %Y %Z"))
print('Copyright     : Hongsuk (hongsuk.yi@gmail.com)')

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [ ]:
#sklearn
from sklearn.svm import SVC 
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, accuracy_score,classification_report
from mlxtend.plotting import plot_decision_regions

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam 

## 포팅

In [ ]:
# from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.utils import algorithm_globals
from qiskit_machine_learning.optimizers import SPSA, L_BFGS_B, COBYLA
from qiskit_machine_learning.algorithms import VQC, QSVC, NeuralNetworkClassifier 

from qiskit.primitives import Sampler

from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, ZFeatureMap, EfficientSU2, TwoLocal
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.exceptions import QiskitMachineLearningError
from qiskit_machine_learning.utils import algorithm_globals

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_orig = pd.read_csv("https://raw.githubusercontent.com/hongsukyi/Lectures/main/data/vds16_L3_5m_Days20.csv")
#df = pd.read_csv("./vds16_L3_120m_Days20.csv")

## Feature selection

In [ ]:
df_orig.head()

In [ ]:
# 각 라벨별 샘풀링 개수 : 200개
sample_per_label = 50

In [ ]:
#각 라벨별로 200개씩 샘플링
df = df_orig.groupby('label').apply(lambda x: x.sample(n=sample_per_label, random_state=42)).reset_index(drop=True)

In [ ]:
# 결과 확인
label_counts = df['label'].value_counts()
print(label_counts)

In [ ]:
# pandas plot을 사용하여 bar chart 그리기
label_counts.plot(kind='bar', title='Number of Samples per Label')
plt.xlabel('Label')
plt.ylabel('Number of Samples')
plt.xticks(rotation=0) # x축 라벨 회전 (필요에 따라 조절)
plt.tight_layout() # 레이아웃 조정
plt.show()

In [ ]:
df.columns

In [ ]:
sns.scatterplot(df, x = 'ToVol', y = 'Speed', hue='label') 

In [ ]:
sns.lmplot( x = 'rho', y = 'Speed', hue='label', data=df)

# Classical classifiers 

In [ ]:
df.columns

In [ ]:
# classical SVM
X = df[['ToVol', 'Speed','Hour','rho' ]]
y = df['label']

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled.shape)
print(X_scaled[:5,:])

In [ ]:
y_enc = LabelEncoder().fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_enc, test_size=0.20)

In [ ]:
X_test2F=X_test[:, :2]
X_test2F.shape

In [ ]:
# confusion matrix of SVM
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(5,4)): 
    if ymap is not None: 
        y_pred = [ymap[yi] for yi in y_pred] 
        y_true = [ymap[yi] for yi in y_true] 
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels) 
    cm_sum = np.sum(cm, axis=1, keepdims=True) 
    cm_perc = cm / cm_sum.astype(float) * 100 
    annot = np.empty_like(cm).astype(str) 
    nrows, ncols = cm.shape 
    for i in range(nrows): 
        for j in range(ncols): 
            c = cm[i, j] 
            p = cm_perc[i, j] 
            if i == j: 
                s = cm_sum[i] 
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s) 
            elif c == 0: 
                annot[i, j] = '' 
            else: 
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels) 
    cm.index.name = 'Actual' 
    cm.columns.name = 'Predicted' 
    fig, ax = plt.subplots(figsize=figsize) 
    sns.heatmap(cm, annot=annot, fmt='', ax=ax) 

## SVM

In [ ]:
acc_sv=0.0
acc_ann=0.0
acc_dnn=0.0
acc_qsvc=0.0
acc_vqc=0.0

In [ ]:
result=[]

In [ ]:
# Create svm Classifier
model = SVC()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

a_svm =  model.score(X_test,y_test) 
result.append(a_svm) 
print("Test accuracy_score:%.4f"%accuracy_score(y_test, y_pred))
print("Confustion Matrix:\n",confusion_matrix(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test, y_pred))

In [ ]:
 cm_analysis(y_test, y_pred, labels=None, ymap=None, figsize=(8,5)) 

In [ ]:
model.fit(X_test2F, y_test)

plot_decision_regions(X_test2F, y_test, clf=model)
plt.xlabel('Traffic Volume')
plt.ylabel('Traffic Speed')
plt.title('SVM Decision Boundary')
plt.show()

## ANN

In [ ]:
# classical QNN 
model= MLPClassifier(max_iter=1000,  random_state=5)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
a_ann =  model.score(X_test,y_test) 
result.append(a_ann) 
print("Test accuracy_score:%.4f"%accuracy_score(y_test, y_pred))
print("Confustion Matrix:\n",confusion_matrix(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test, y_pred))

In [ ]:
cm_analysis(y_test, y_pred,labels=None, ymap=None, figsize=(8,6)) 

In [ ]:
model.fit(X_test2F, y_test)
plot_decision_regions(X_test2F, y_test, clf=model)
plt.xlabel('Occupancy')
plt.ylabel('Traffic Volume')
plt.title('ANN Decision Boundary')
plt.show()

## Deep Neural Network MLP

In [ ]:
num_classes = y.nunique()
num_features = X_train.shape[1]
print('특성 개수  :',num_features)
print(f"클래스 개수: {num_classes}")

In [ ]:
model = Sequential([
    Dense(64, input_dim=num_features, activation = 'relu'),
    Dense(32, activation = 'relu'),
    Dense(num_classes, activation= 'softmax')])  

dnn2F = Sequential([
    Dense(64, input_dim= 2, activation = 'relu'),
    Dense(32, activation = 'relu'),
    Dense(num_classes, activation= 'softmax')])  

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train,y_train, epochs=50, batch_size=32, validation_split=0.2,verbose=0)

In [ ]:
_, a_dnn = model.evaluate(X_test,y_test)

In [ ]:
print("Accuracy of DNN:%.4f"%a_dnn)
result.append(a_dnn) 

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train_loss', color='blue')
plt.plot(history.history['val_loss'], label='val_loss', color='red')

# 현재 axes 객체 가져오기
ax1 = plt.gca()
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')

# x축을 공유하는 새로운 axes 객체 생성 (오른쪽 y축)
ax2 = ax1.twinx()
ax2.plot(history.history['accuracy'], label='train_accuracy', color='green')
ax2.plot(history.history['val_accuracy'], label='val_accuracy', color='lime')
ax2.set_ylabel('Accuracy')

# 범례를 명시적으로 처리 (두 개의 axes에 대한 라벨을 합치기)
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='center right')

plt.title('Model Training History')
plt.show()

In [ ]:
class Onehot2Int(object):
    def __init__(self, model):
        self.model = model
    def predict(self, X):
        y_pred = self.model.predict(X)
        return np.argmax(y_pred, axis=1)   

model =Onehot2Int(dnn2F)

ax=plot_decision_regions(X_test2F, y_test, clf=model)
plt.xlabel('Traffic Volume')
plt.ylabel('Traffic Speed')
plt.title('DNN : Decision Boundary')

handles, labels = ax.get_legend_handles_labels()
if num_classes == 3 : 
    ax.legend(handles, ['Fast', 'Show', 'Jam'],  framealpha=0.3, scatterpoints=1)
plt.show()


# Quantum Simulator for Quantum Classification

In [ ]:
import time
from qiskit_machine_learning.algorithms import QSVC, VQC
from qiskit.circuit.library import ZZFeatureMap,  RealAmplitudes
from qiskit_machine_learning.optimizers import COBYLA
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit.primitives import StatevectorSampler as Sampler

### 1) VQC

feature map
- from classical data to qubit through data encoding, data embedding, data loading
- The most standard fearture is ZZFeatureMap

ansatz

- apply a parameterize quantum circuit.
- objective function characterized the distance between the prediction and known labeled data.
- A parametrized quantum circuit is call a paramenterized trial state, variational form, or ansatz.- 

## 수정할 부분

In [ ]:
feature_map = ZZFeatureMap(feature_dimension = num_features , reps=1, entanglement="linear")
ansatz = RealAmplitudes(num_qubits = num_features , reps=1, entanglement="linear")
#ansatz.decompose().draw(output="mpl", style="clifford", fold=20)
#ansatz.decompose().draw(output="mpl", style="clifford", fold=20)
#feature_map.decompose().draw(output="mpl", fold=20)   #style="clifford",  fold=18

StatevectorSampler를 이용한 Simulator

In [ ]:
sampler = Sampler()

In [ ]:
def multi_class_interpret(measured_integer):
    """측정된 정수를 num_classes으로 나눈 나머지를 클래스 인덱스로 반환"""
    return measured_integer % num_classes
    
# callback function that draws a live plot when the .fit() method is called
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

Onh-hot encoded format의 경우 crossEntropyLoss를 사용한다.

In [ ]:
from qiskit_machine_learning.optimizers import COBYLA, L_BFGS_B, ADAM, SLSQP, AQGD
optimizer = COBYLA(maxiter=80)

In [ ]:
from qiskit_machine_learning.algorithms.classifiers import VQC

vqc = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    callback=callback_graph,
    output_shape=num_classes,
    interpret=multi_class_interpret  # interpret 함수 추가
)

In [ ]:
plt.rcParams["figure.figsize"] = (12, 6)

import time
# clear objective value history

objective_func_vals = []
start = time.time()
vqc.fit(X_train, y_train)
elapsed = time.time() - start

print(f"Training time: {round(elapsed)} seconds")

In [ ]:
#plt.plot(objective_func_vals)

In [ ]:
train_score_q4 = vqc.score(X_train, y_train)
a_vqc = vqc.score(X_test, y_test)

print(f"Quantum VQC on the training dataset: {train_score_q4:.2f}")
print(f"Quantum VQC on the test dataset    : {a_vqc:.2f}")

VQC performance

In [ ]:
y_pred = vqc.predict(X_test)

# print classification report and confusion matrix for the classifier
print("Classification report: \n", metrics.classification_report(y_test, y_pred))
print("Confusion matrix: \n", metrics.confusion_matrix(y_test, y_pred))

In [ ]:
predict = vqc.predict(X_test)
print(f"Predicted labels: {predict}")
print(f"Ground truth:     {y_test}")

In [ ]:
df = pd.DataFrame({
    'models': ['SVC', 'ANN','DNN', 'VQC'],
    'accuracy':[a_svm, a_ann, a_dnn,  a_vqc]})
df=df.sort_values(by='accuracy', ascending=True)

df.plot.bar(x='models',y='accuracy',rot=15)

In [ ]:
fm_qsvc = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=2, entanglement="linear")
fm_qsvc.decompose().draw(output="mpl", style="clifford", fold=20)
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
qsvc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=fm_qsvc)
qsvc = QSVC(quantum_kernel=qsvc_kernel)

In [ ]:
start = time.time()
qsvc.fit(X_train, y_train)
elapsed = time.time() - start
print(f"Training time: {round(elapsed)} seconds")

In [ ]:
a_qsvc = qsvc.score(X_test, y_test)
print(f"QSVC classification test score: {a_qsvc}")

In [ ]:
df = pd.DataFrame({
    'models': ['SVC', 'ANN','DNN','Q_SVC', 'Q_VQC'],
    'accuracy':[acc_svc, acc_ann, acc_dnn, acc_qsvc, acc_vqc]})
df=df.sort_values(by='accuracy', ascending=True)

df.plot.bar(x='models',y='accuracy',rot=15)

In [ ]:
# classification report of QSVC
y_pred = qsvc.predict(X_test) 
metrics.classification_report(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)